In [172]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [173]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [174]:
from ml.models.base.tabular_dataset import TabularDataset
from ml.preprocess import data

In [178]:
from torch.utils.data import random_split, DataLoader, Dataset

import torch
# from torch.nn import functional as F
from torchmetrics import Accuracy, MeanSquaredError
from torch import nn
import pytorch_lightning as pl
from sklearn.metrics import mean_squared_error
import pandas as pd

In [176]:
from sklearn.model_selection import train_test_split

X_train_orig, y = data.get_dataset(f_name = '20220319_covid_merge_processed.csv' , sep = ',')
# X_tilda = X.drop(brisk_xgboost.DROP_LIST, axis = 1)
X = X_train_orig.drop(['location'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)


In [130]:

class TabularDataset(Dataset):    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __getitem__(self, index):        
        # num_array = torch.tensor(num_array, dtype = torch.float32)
        # num_array = torch.tensor(self.x.loc[index], dtype=torch.float32)        
        num_array = torch.tensor(self.x.iloc[index], dtype=torch.float32)        
        # label_array = self.df[self.label_col].iloc[idx]
        label_array = torch.tensor(self.y.iloc[index], dtype=torch.float32)
        # label_array = label_array.reshape(self.x[0], )
        return num_array, label_array

    def __len__(self):
        return len(self.x)

In [169]:
from dask_ml.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

class BriskDataModule(pl.LightningDataModule):
    def __init__(self, X, y, batch_size):
        #Define required parameters here
        super().__init__()
        self.X = X
        self.y = y
        self.TEST_SIZE = 0.3
        self.VAL_SIZE = 0.2
        self.BATCH_SIZE = batch_size
        self.SEED = 42
        # self.train_dataset = None
        # self.test_dataset = None

        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None
        self.pred_dataset = None

        # self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, random_state=self.SEED, test_size=self.TEST_SIZE)

        X_train, self.X_test, y_train, self.y_test = train_test_split(self.X, self.y, random_state=self.SEED,
                                                                                                 shuffle=True, test_size=self.TEST_SIZE)
    
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(X_train, y_train, random_state=self.SEED,
                                                                                                 shuffle=True, test_size=self.VAL_SIZE)


        
    def setup(self, stage=None):
        # Define steps that should be done on 
        # every GPU, like splitting data, applying
        # transform etc.
        # generate indices: instead of the actual data we pass in integers instead
        # print(self.test_split)

        # self.X = StandardScaler().fit_transform(self.X)


        # print(X_train.shape)
        # self.train_dataset = TabularDataset(X_train, y_train)
         #self.test_dataset = TabularDataset(X_test, y_test)
 
        # self.train_split = DataLoader(cdataset, batch_size=8, shuffle=True)

        if stage == "fit":
            self.train_dataset = TabularDataset(self.X_train, self.y_train)
            self.val_dataset = TabularDataset(self.X_val, self.y_val)
            # print('fit')


        # Assign test dataset for use in dataloader(s)
        if stage == "test":
            self.test_dataset = TabularDataset(self.X_test, self.y_test)
            # print('test')
            

        if stage == "predict":
            self.pred_dataset = TabularDataset(self.X_test, self.y_test)
            # print('predict')
        


    def train_dataloader(self):
        # Return DataLoader for Training Data here
        # print('train_loader')        
        return DataLoader(self.train_dataset, batch_size=self.BATCH_SIZE, shuffle=False)


    def val_dataloader(self):
        # Return DataLoader for Testing Data here
        # print('val_dataloader')
        dl = DataLoader(self.val_dataset, batch_size=self.BATCH_SIZE, shuffle=False)
        return dl

        
    def test_dataloader(self):
        # Return DataLoader for Testing Data here
        # print('test_dataloader')
        return DataLoader(self.test_dataset, batch_size=self.BATCH_SIZE, shuffle=False)


    def predict_dataloader(self):
        # print('predict_dataloader')
        return DataLoader(self.pred_dataset, batch_size=self.BATCH_SIZE)


In [164]:
class BriskModel(pl.LightningModule): 
    def __init__(self, input_dim = 40): 
        super(BriskModel, self).__init__() 

        # Defining learning rate
        self.lr = 0.004
        #self.batch_size = batch_size
        #self.X = X
        #self.y = y
        #hidden_size = X.shape[0]

        # Defining our model architecture
        self.layer_0 = nn.BatchNorm1d(num_features = input_dim)
        # self.layer_0 = nn.Linear(hidden_size, 200)
        self.layer_1 = nn.Linear(in_features = input_dim, out_features = 115)
        self.layer_2 = nn.Linear(115, 99)
        self.layer_3 = nn.Linear(99, 164)
        self.layer_final = nn.Linear(164, 1)
        self.relu = nn.ReLU()

        self.flatten = torch.nn.Flatten(0, 1)

        # nn.init.xavier_normal_(self.layer_1.weight)
        # self.layer_2 = nn.BatchNorm1d(16)
        # self.layer_3 = nn.Linear(16, 96)
        # nn.init.xavier_normal_(self.layer_3.weight)
        # self.layer_4 = nn.Linear(96, 1)
        # nn.init.xavier_normal_(self.layer_4.weight)
        # self.out = nn.Flatten(0, 1)
                  
          
        # Defining loss 
        ##self.loss = nn.BCEWithLogitsLoss()
        # self.loss = nn.MSELoss()
    
        self.val_accuracy = Accuracy()
        self.test_accuracy = Accuracy()

        self.val_mse = MeanSquaredError()
        self.train_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()

        self.predictions = []


    def forward(self, x):
#        return self.model(x)

        # Defining the forward pass of the model
        # print(x)
        # batch_size, _= x.size() 
        # x = x.view(batch_size, -1) 
        
        # Pass the tensor through the layers
        #x = self.layer_1(x)
        #x = nn.relu(x)
        #x = self.layer_2(x)
        #x = self.layer_3(x)
        #x = F.relu(x)
        #x = self.layer_4(x)
        #x = F.sigmoid(x)
        #x = self.out(x)
        #return x

        x = self.relu(self.layer_0(x))
        x = self.relu(self.layer_1(x))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x = self.flatten(self.layer_final(x))
        return x
    
    def configure_optimizers(self):
          # Defining and returning the optimizer for our model
        # with the defines parameters
        #return torch.optim.SGD(self.parameters(), lr = self.lr) 
        return torch.optim.Adam(self.parameters(), lr=self.lr)
    
    def training_step(self, train_batch, batch_idx): 
        # Defining training steps for our mode
        x, y = train_batch        
        y = y.reshape(y.shape[0], )

        pred = self.forward(x)        

        #print(pred.shape)
        # print(y.shape)

        # mean_squared_error(pred, y)

        loss = nn.MSELoss()(pred, y)

        # print(f'train los:{loss}')

        # self.log("my_loss", loss, on_step=True, on_epoch=True, prog_bar=False, logger=True)

        self.train_mse.update(pred, y)


        return loss

        x, y = train_batch
        logits = self.forward(x)#.to(torch.float32)
        
        logits = torch.round(torch.sigmoid(logits))
        
        # logits = logits.to(torch.float32)
        y = y.to(torch.float32)
        # print('train_y:' + str(len(y)) + ' train_prec_y:' + str(len(logits)))
        # print('train:' + str(max(logits)) + ' min:' + str(min(logits)))
        loss = self.loss(logits, y)
        
        bin_acc = binary_acc(logits, y)  
        
        
        return loss


    def on_train_end(self) -> None:
        print(f'training ended: MSE {self.train_mse.compute()}')
        return super().on_train_end()



    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.reshape(y.shape[0], )
        pred = self.forward(x)
        

        loss = nn.MSELoss()(pred, y)

        # print(f'val loss {loss}')
        # self.val_accuracy.update(preds, y)

        self.val_mse.update(pred, y)
#       

        # Calling self.log will surface up scalars for you in TensorBoard
        #self.log("val_loss", loss, prog_bar=True)
        # self.log("val_acc", self.val_accuracy, prog_bar=True)
        return loss

    

    def on_validation_end(self) -> None:
        print(f'validation ended: MSE {self.val_mse.compute()}')
        return super().on_validation_end()


    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        y = y.reshape(y.shape[0], )
        pred = self.forward(x)
        loss = nn.MSELoss()(pred, y)

        self.test_mse.update(pred, y)

        list_np = pred.cpu().detach().numpy()

        self.predictions.extend(list_np)
        # preds = torch.argmax(logits, dim=1)
        #self.test_accuracy.update(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        #self.log("test_loss", loss, prog_bar=True)
        #self.log("test_acc", self.test_accuracy, prog_bar=True)
        return loss



    def on_test_end(self) -> None:
        print(f'test ended: MSE {self.test_mse.compute()}')
        return super().on_test_end()

    def on_predict_end(self) -> None:
        
        return super().on_predict_end()

    ####################
    # DATA RELATED HOOKS
    ####################

    # def prepare_data(self):
    #     # download
    #     self.TEST_SIZE = 0.3
    #     self.VAL_SIZE = 0.2
    #     self.SEED = 42
    #     self.train_dataset = None
    #     self.test_dataset = None
    #     self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, random_state=self.SEED, test_size=self.TEST_SIZE)

    #     self.train_dataset = TabularDataset(self.X_train, self.y_train)
    #     self.test_dataset = TabularDataset(self.X_test, self.y_test)


    #def train_dataloader(self):
     
    #     return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    # def val_dataloader(self):
    #     return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=True)

    # def test_dataloader(self):
    #     return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=True)
        



In [165]:
clf = BriskModel(input_dim = 40)
X_scalar = StandardScaler().fit_transform(X)
data = BriskDataModule(X_scalar, y, batch_size=8)
# data.setup()
trainer = pl.Trainer(min_epochs=10, max_epochs=15, enable_progress_bar=False)
trainer.fit(clf, data)

validation ended: MSE 3367578.0
validation ended: MSE 2784127.75
validation ended: MSE 2362753.25
validation ended: MSE 2036817.0
validation ended: MSE 1825850.625
validation ended: MSE 1690776.5
validation ended: MSE 1623812.0
validation ended: MSE 1613365.875
validation ended: MSE 1654475.875
validation ended: MSE 1741227.875
validation ended: MSE 1864921.75
validation ended: MSE 2010703.375
validation ended: MSE 2165355.5
validation ended: MSE 2323644.25
validation ended: MSE 2480604.75
validation ended: MSE 2636054.25
training ended: MSE 674998.875


In [166]:
trainer.validate(clf, data)

validation ended: MSE 2772766.0


[{}]

In [167]:
trainer.test(clf, data)

test ended: MSE 1159812.375


[{}]

In [170]:
clf.predictions

[642.51807,
 22.145275,
 920.18005,
 885.9208,
 1534.6659,
 119.19596,
 1084.5883,
 46.34888,
 1012.996,
 1562.5753,
 2141.5884,
 1670.9703,
 3010.036,
 863.91846,
 58.483875,
 2410.4988,
 749.8333,
 39.044827,
 1634.5511,
 826.71204,
 488.48053,
 237.09915,
 157.33914,
 1438.2742,
 493.89413,
 878.25464,
 502.72635,
 406.95267,
 734.8441,
 2244.5544,
 3273.3074,
 846.9385,
 191.31969,
 2687.222,
 1508.3539,
 1652.4308,
 368.1294,
 2151.7327,
 116.3273,
 96.399956,
 1115.7792,
 22.405722,
 586.69684,
 1858.5703,
 1574.1672,
 1529.2097,
 1093.684,
 58.240833,
 2192.2852,
 1058.5497,
 795.8325,
 819.9603,
 1591.4465,
 94.195015,
 109.405045,
 944.9503,
 2539.932,
 1346.3926]

In [179]:
df = pd.DataFrame(list(zip(X_train_orig['location'].values, y_test.values.reshape(48, ), clf.predictions )),
               columns =['countary', 'deaths', 'predictions'])
df



,countary,deaths,predictions
0,0,0.000,642.518066
1,1,4896.644,22.145275
2,2,1486.844,920.180054
3,3,55.799,885.920776
4,4,153.473,1534.665894
5,5,2252.476,119.195961
6,6,146.353,1084.588257
7,7,0.000,46.348881
8,8,604.743,1012.995972
9,9,45.966,1562.575317


In [181]:
briskmodel_loaded = BriskModel(input_dim = 40)

briskmodel_loaded.load_from_checkpoint(checkpoint_path='./lightning_logs/version_42/checkpoints/epoch=14-step=210.ckpt')

# clf.load_from_checkpoint()

BriskModel(
  (layer_0): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_1): Linear(in_features=40, out_features=115, bias=True)
  (layer_2): Linear(in_features=115, out_features=99, bias=True)
  (layer_3): Linear(in_features=99, out_features=164, bias=True)
  (layer_final): Linear(in_features=164, out_features=1, bias=True)
  (relu): ReLU()
  (flatten): Flatten(start_dim=0, end_dim=1)
  (val_accuracy): Accuracy()
  (test_accuracy): Accuracy()
  (val_mse): MeanSquaredError()
  (train_mse): MeanSquaredError()
  (test_mse): MeanSquaredError()
)